In [1]:
import warnings 

warnings.simplefilter("ignore")

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Según distintos benchmarks (papers, kaggle.com) los **algoritmos de uso general** que tienen más veces la mejor performance son: **Gradient Boosted Trees, Random Forest y SVM**.

Decision Trees

In [3]:
from sklearn.tree import DecisionTreeRegressor 

model = DecisionTreeRegressor(max_depth=4)

In [4]:
X = pd.read_csv('X_open.csv')
y = X['worldwide_gross']

X = X.drop('worldwide_gross', axis=1)

In [5]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [6]:
model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

Virtudes de los arboles de decision:

Metodo poderoso y probado.

Interpretable.

No necesita escalar los datos (clasificación), y menos preprocesamiento de variables.

Sin embargo en la practica existen modelos que obtienen mejor rendimiento. Como mejorar el modelo de arboles de decisión?.

Ensembles
Concepto General

Random Forest y Gradient Boosted Trees, forman parte de una familia de algoritmos que se denominan ensembles.

$$ Ensemble = Submodelos \rightarrow Entrenamiento \rightarrow Predicciones_{Intermedias} \rightarrow Voto \rightarrow Prediccion_{final}$$
Cómo funciona el algoritmo Random Forest?

Vamos a generar cientos de modelos de arboles de decisión que serán entrenados sobre conjuntos de datos bootstrapeados del conjunto de datos original y donde para cada etapa de separación el conjunto de features elegibles sera un subconjunto aleatorio del conjunto original de features.



Cada uno de los arboles entrenados luego podrá votar por su predicción y promediaremos estos votos.


Ensembles del pobre ("Poor man's ensembles")

Entrenar diversos modelos a mano.

Promediar el resultado.

Owen Zhang, número 1 de Kaggle.com durante un largo tiempo, ocupaba esta estrategia promediando diversos modelos XGBoost.

from sklearn.ensemble import VotingClassifier sirve por ejemplo para hacer un ensemble manual de clasificación.


En general los ensembles del pobre funcionan ya que cada uno de los modelos que votarán en conjunto son bastante fuertes.



Porqué RF es poderoso?

**Leo Breiman** creador del Random Forest demostró que un ensemble podía tener buen poder de generalización sí:

Los submodelos tienen buen poder de predicción.

Los submodelos están descorrelacionados.

Así el algoritmo de Random Forest compromete un poco de poder de predicción de cada uno de los decision trees que arma, pero la forma aleatoria de generarlos hace que esten fuertemente descorrelacionados.


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

forest = RandomForestRegressor(200)
results = cross_validate(forest,X,y,cv=5,scoring='r2', return_train_score=True)

In [10]:
test_scores = results['test_score']
train_scores = results['train_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

0.9654064427944192
0.5179828852650891


Mejor resultado que Lasso! Ya no tenemos Bias y tenemos un mejor score r2. Sin embargo tenemos una diferencia importante entre score de entrenamiento y de test (overfit).



Gradient Boosted Trees

In [11]:
from sklearn.ensemble import GradientBoostingRegressor

ensemble = GradientBoostingRegressor()
results = cross_validate(ensemble, X, y, cv=5, scoring='r2', return_train_score=True)

In [12]:
test_scores = results['test_score']
train_scores = results['train_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

0.9151392143549633
0.5252396990065117


Cómo optimizamos los parametros de este último modelo?

Optimización de hiperparametros.

Fijar un learning rate alto.

Fijar parametros de los arboles.

Fijados estos parametros, elegir el mejor numero de estimadores que conforman el ensemble.


Por ahora dijimos que:

train_test_split servia para evaluaciones rapidas, testeos y prototipaje.

cross_validate es un método más robusto para poder estimar el rendimiento de tu algoritmo.

Sin embargo una vez que hemos finalizado nuestra etapa de prototipaje y ya queremos establecer un modelo definitivo deberiamos seguir el flujo siguiente.

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [14]:
len(X_train)

1665

In [15]:
len(y_train)

1665

In [16]:
from sklearn.model_selection import GridSearchCV

param_test1 = {'n_estimators': range(20,501,20)}

In [17]:
list(param_test1['n_estimators'])

[20,
 40,
 60,
 80,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 440,
 460,
 480,
 500]

In [18]:
estimator = GradientBoostingRegressor(learning_rate=0.1,
                                     min_samples_split=500,
                                     min_samples_leaf=50,
                                     max_depth=8,
                                     max_features='sqrt',
                                     subsample=0.8,
                                     random_state=10)

In [19]:
gsearch1 = GridSearchCV(estimator,
                        param_grid= param_test1,
                       scoring='r2',
                       cv=5, return_train_score=True)

In [20]:
gsearch1.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=8,
                                                 max_features='sqrt',
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=50,
                                                 min_samples_split=500,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change=None,
           

In [21]:
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([0.02861829, 0.05789857, 0.07839999, 0.10439763, 0.13166389,
         0.15975361, 0.18995752, 0.20518484, 0.22954178, 0.25570235,
         0.28561726, 0.31275535, 0.33076105, 0.36337562, 0.38284912,
         0.41145406, 0.44519076, 0.46259527, 0.50966349, 0.52382665,
         0.55142059, 0.57065482, 0.59955378, 0.63477273, 0.65585127]),
  'std_fit_time': array([0.00248547, 0.00574512, 0.00058447, 0.01014748, 0.00632882,
         0.00732344, 0.00781031, 0.00386627, 0.0068016 , 0.00544892,
         0.00374142, 0.00064401, 0.00697345, 0.00580139, 0.00704698,
         0.00536571, 0.00777551, 0.00924505, 0.01994091, 0.00834598,
         0.004308  , 0.00499817, 0.0058235 , 0.01697258, 0.00704614]),
  'mean_score_time': array([0.00183158, 0.        , 0.        , 0.00685654, 0.00245061,
         0.0042872 , 0.00525441, 0.00124168, 0.00312099, 0.00632873,
         0.00655847, 0.00312152, 0.00936437, 0.00312386, 0.0064867 ,
         0.00936437, 0.00315366, 0.01249309, 0.

In [22]:
final_results = cross_validate(gsearch1.best_estimator_,X_train,y_train,return_train_score=True)

In [23]:
test_scores = final_results['test_score']
train_scores = final_results['train_score']
print(np.mean(train_scores))
print(np.mean(test_scores))


0.8183169366503803
0.74138761251573


In [24]:
estimator = GradientBoostingRegressor(learning_rate=0.1,
                                     min_samples_split=500,
                                     min_samples_leaf=50,
                                     max_depth=8,
                                     max_features='sqrt',
                                     subsample=0.8,
                                     random_state=10,
                                     n_estimators=240)

In [25]:
estimator.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=8,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=50, min_samples_split=500,
                          min_weight_fraction_leaf=0.0, n_estimators=240,
                          n_iter_no_change=None, presort='auto',
                          random_state=10, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [26]:
estimator.score(X_test,y_test)

0.8092888852563106